In [6]:
import torch
from torch.utils.data import Dataset
import json
import os
import torch.nn as nn
import torch.nn.functional as func
from torch.utils.data import DataLoader
import torch.optim as optim
from os.path import expanduser
import splitfolders
import shutil
import glob
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [8]:
class KpVelDataset(Dataset):
    def __init__(self, json_folder):
        super(KpVelDataset, self).__init__()
        self.data = []
        for json_file in sorted(os.listdir(json_folder)):
            if json_file.endswith('_combined.json'):
                with open(os.path.join(json_folder, json_file), 'r') as file:
                    data = json.load(file)
                    start_kp = data['start_kp']
                    next_kp = data['next_kp']
                    velocity = data['velocity']
                    self.data.append((start_kp, next_kp, velocity))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        start_kp, next_kp, velocity = self.data[idx]
        # Extract and flatten the first two elements of each keypoint in start_kp
        start_kp_flat = torch.tensor([kp for sublist in start_kp for kp in sublist[0][:2]], dtype=torch.float)
        # Extract and flatten the first two elements of each keypoint in next_kp
        next_kp_flat = torch.tensor([kp for sublist in next_kp for kp in sublist[0][:2]], dtype=torch.float)
        velocity = torch.tensor(velocity)
        return start_kp_flat, next_kp_flat, velocity

In [9]:
def train_test_split(src_dir):
#     dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_anno)
        
#     for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
#         shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*_combined.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = src_dir + "split_folder_reg"
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(0.8, 0.1, 0.1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
#     shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class KeypointRegressionNet(nn.Module):
    def __init__(self):
        super(KeypointRegressionNet, self).__init__()
        # Define the architecture
        self.fc1 = nn.Linear(15, 128)  # 18 keypoints + 3 velocity values
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 12)  # Output size is 18 (6 keypoints * 3 values each)

    def forward(self, start_kp, velocity):
        # Flatten start keypoints and concatenate with velocity
        x = torch.cat((start_kp, velocity), dim=1)
        
        # Forward pass through the network
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)     # No activation function in the last layer
        return x

In [11]:
# Initialize dataset and data loader
# to generalize home directory. User can change their parent path without entering their home directory
num_epochs = 300
batch_size = 128
v = 1
layers = 5
root_dir = '/home/jc-merlab/Pictures/panda_data/panda_sim_vel/vel_reg_sim_test_2/'
# print(root_dir)
split_folder_path = train_test_split(root_dir)
dataset = KpVelDataset(root_dir)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize model
model = KeypointRegressionNet()  # Adjust input_size as necessary
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Training loop
for epoch in range(num_epochs):
    for start_kp, next_kp, velocity in data_loader:
        optimizer.zero_grad()
        velocity = velocity.squeeze(1)
        # print(start_kp.shape)
        # print(velocity.shape)
        output = model(start_kp, velocity)
        loss = criterion(output, next_kp)
        loss.backward()
        optimizer.step()
        # print("output", output)
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')
    
# # Save the trained model
model_save_path = f'/home/jc-merlab/Pictures/Data/trained_models/reg_nkp_b{batch_size}_e{num_epochs}_v{v}_l{layers}.pth'
torch.save(model.state_dict(), model_save_path)

# model_save_path = f'/home/jc-merlab/Pictures/Data/trained_models/reg_nkp_b{batch_size}_e{num_epochs}_v{v}.pth'
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'model_structure': KeypointRegressionNet()
# }, model_save_path)
# print(f"Model saved to {model_save_path}")


Copying files: 3137 files [00:00, 19419.51 files/s]


Epoch 1, Loss: 2981.0869140625
Epoch 2, Loss: 2047.0892333984375
Epoch 3, Loss: 613.0527954101562
Epoch 4, Loss: 178.31964111328125
Epoch 5, Loss: 132.6424560546875
Epoch 6, Loss: 121.27253723144531
Epoch 7, Loss: 75.35826110839844
Epoch 8, Loss: 62.921722412109375
Epoch 9, Loss: 60.99764633178711
Epoch 10, Loss: 59.945953369140625
Epoch 11, Loss: 53.35378646850586
Epoch 12, Loss: 40.25061798095703
Epoch 13, Loss: 42.52675247192383
Epoch 14, Loss: 45.38826370239258
Epoch 15, Loss: 31.490921020507812
Epoch 16, Loss: 121.34574890136719
Epoch 17, Loss: 27.4359188079834
Epoch 18, Loss: 22.975778579711914
Epoch 19, Loss: 21.23049545288086
Epoch 20, Loss: 22.396955490112305
Epoch 21, Loss: 33.31021499633789
Epoch 22, Loss: 24.628881454467773
Epoch 23, Loss: 16.68520736694336
Epoch 24, Loss: 32.51499938964844
Epoch 25, Loss: 36.69099426269531
Epoch 26, Loss: 24.239931106567383
Epoch 27, Loss: 14.51125717163086
Epoch 28, Loss: 14.571964263916016
Epoch 29, Loss: 21.00263023376465
Epoch 30, Loss

Epoch 237, Loss: 2.7620599269866943
Epoch 238, Loss: 1.4841718673706055
Epoch 239, Loss: 3.39276123046875
Epoch 240, Loss: 1.3663345575332642
Epoch 241, Loss: 9.696428298950195
Epoch 242, Loss: 1.1811614036560059
Epoch 243, Loss: 16.924360275268555
Epoch 244, Loss: 6.8007025718688965
Epoch 245, Loss: 4.27230167388916
Epoch 246, Loss: 1.8754184246063232
Epoch 247, Loss: 1.8390531539916992
Epoch 248, Loss: 3.4073126316070557
Epoch 249, Loss: 7.869102478027344
Epoch 250, Loss: 16.36129379272461
Epoch 251, Loss: 7.159234523773193
Epoch 252, Loss: 2.682500123977661
Epoch 253, Loss: 0.8183968663215637
Epoch 254, Loss: 3.935009002685547
Epoch 255, Loss: 0.9532617926597595
Epoch 256, Loss: 8.297929763793945
Epoch 257, Loss: 6.1421074867248535
Epoch 258, Loss: 2.0758914947509766
Epoch 259, Loss: 22.533109664916992
Epoch 260, Loss: 10.462942123413086
Epoch 261, Loss: 1.6063036918640137
Epoch 262, Loss: 2.4094417095184326
Epoch 263, Loss: 5.967643737792969
Epoch 264, Loss: 1.1284935474395752
Epoc